## Load modules

In [1]:
push!(LOAD_PATH, "./dvi/src")

addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using SCAs, SCAIterators, SCAConst
using GridInterpolations

mdp = SCA()

SCA(5446,36,StateIterator([-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[0.0,1.5708,3.14159,4.71239,6.28319],[10.0,15.0,20.0],[10.0,15.0,20.0],1,1,1,1,1,5446),ActionIterator([:right20,:right10,:straight,:left10,:left20,:clearOfConflict],1,1,36),RectangleGrid with 5445 points)

## Check size of MDP

In [2]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println("mdp of type ", typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

mdp of type SCA takes up 1.768 kB


## Informal validation of state and action iterators and transition function

In [3]:
testActions = ActionIterator(Actions)
actionCount = 0
for action in testActions
    actionCount += 1
    println("action #", actionCount, ": ", typeof(action), " = ", action)
end # for action
println("nActions = ", NActions)
println("actionCount = ", actionCount, "\n")

testStates = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in testStates
    stateCount += 1
    if stateCount == 1 || stateCount == NStates
        println("state #", stateCount, ": ", typeof(state), " = ", state)
    end # if
end # for state
println("nStates = ", NStates)
println("stateCount = ", stateCount)

action #1: Action = Action(:right20,:right20)
action #2: Action = Action(:right10,:right20)
action #3: Action = Action(:straight,:right20)
action #4: Action = Action(:left10,:right20)
action #5: Action = Action(:left20,:right20)
action #6: Action = Action(:clearOfConflict,:right20)
action #7: Action = Action(:right20,:right10)
action #8: Action = Action(:right10,:right10)
action #9: Action = Action(:straight,:right10)
action #10: Action = Action(:left10,:right10)
action #11: Action = Action(:left20,:right10)
action #12: Action = Action(:clearOfConflict,:right10)
action #13: Action = Action(:right20,:straight)
action #14: Action = Action(:right10,:straight)
action #15: Action = Action(:straight,:straight)
action #16: Action = Action(:left10,:straight)
action #17: Action = Action(:left20,:straight)
action #18: Action = Action(:clearOfConflict,:straight)
action #19: Action = Action(:right20,:left10)
action #20: Action = Action(:right10,:left10)
action #21: Action = Action(:straight,:left1

In [5]:
nextStateIndices, probs = nextStates(mdp, 1, 15)
println("next state indices:\n", nextStateIndices, "\n")
println("probabilities:\n", probs)

next state indices:
[1,1,2,5446,5446,1,2,1816,1817,5446,5446,5446,5446]

probabilities:
[0.3333333333333333,0.08125,0.0020833333333333355,0.08333333333333334,0.08333333333333334,0.04875,0.0012500000000000013,0.03250000000000001,0.0008333333333333341,0.08333333333333334,0.08333333333333334,0.08333333333333334,0.08333333333333334]


## Parallel solution

In [6]:
numProcs = int(CPU_CORES / 2)
solver = ParallelSolver(
    numProcs,
    maxIterations = 1000,
    tolerance = 1e-6,
    gaussSiedel = false,
    includeV = true,
    includeQ = true,
    includeA = true)

ParallelSolver(2,None[],1000,1.0e-6,false,true,true,true)

In [ ]:
policy = solve(solver, mdp, verbose = true)
println("")

## Check against reference solution by visual inspection

In [ ]:
push!(LOAD_PATH, "../src/")
using Pairwise, Multiagent, HDF5, JLD

d = get_pomdp()
refQ = load("../data/alpha.jld", "alpha")
solQ = policy.Q'
save("../data/par-alpha.jld", "solQ", solQ)

### Reference policy plot

In [ ]:
viz_pairwise_policy(d, refQ)

### Parallel solver policy plot

Note: We haven't implemented sigma-sampling.

In [ ]:
function sharray2array(sharray::SharedArray{Float64, 2})
    result = zeros(solQ.dims)
    for i = 1:solQ.dims[1]
        for j = 1:solQ.dims[2]
            result[i, j] = sharray[i, j]
        end # for j
    end # for i
    return result
end # function sharray2array

In [ ]:
viz_pairwise_policy(d, sharray2array(solQ))